# Training of neural networks

To train neural networks, we first need to load the dataset. If you don't know how `AirfoilDataset` and `FileDataFiles` works, please refer to  `process_dataset.ipynb`.
The following codes showcase how to load a dataset that is used in the single-parameter test in our manuscript.

In [11]:
import zipfile
from airfoil_diffusion.airfoil_datasets import *
from airfoil_diffusion.networks import *
from airfoil_diffusion.trainer import *

if not os.path.exists("./datasets/1_parameter/data/"):
    files=[file for file in os.listdir("./datasets/1_parameter/") if file.endswith(".zip")]
    for file in tqdm(files): 
        f=zipfile.ZipFile("./datasets/1_parameter/"+file,'r')
        for file in f.namelist():
            f.extract(file,"./datasets/1_parameter/data/")
        f.close() 

train_dataset = AirfoilDataset(FileDataFiles("./datasets/1_parameter/train_cases.txt",base_path="./datasets/1_parameter/data/"),
                               data_size=32)

Loading data: 100%|██████████| 125/125 [00:00<00:00, 459.91it/s]


Then we can import our network, `AifNet`. You can use `show_config_options` function to list all the possible configurations of the network:

In [12]:
AifNet().show_config_options()

Mandatory Configuration:
    dim_in (int): The input dim of the model.
    dim_out (int): The output dim of the model.
    dim_basic (int): The basic dimensions in each layer. The real dimension numbers are dim_basic$\times$dim_multipliers.
    dim_multipliers (list): A list used to control the depth and the size of the net. There will be len(dim_multipliers)-1 down/up blocks in the net. The number of input/output channels of each block will also be determined by the elements of this list(dim_basic$\times$dim_multipliers). For instance, if the dim_multipliers is [1 2 4 8]. There will be 3 down/up blocks. The input/output channel of these blocks are (dim_basic, 2$\times$dim_basic), (2$\times$dim_basic, 4$\times$dim_basic) and (4$\times$dim_basic, 8$\times$dim_basic). The size of neckblock will be  8$\times$dim_basic $\times$ input_channel/$2^3$ $\times$ input_channel/$2^3$. If the first elements is 0, the input channel of the first down layer will be the dim_in and the output channel of

All the configurations can be specified either by a YAML configuration file or by directly inputting when creating the network. If you give both the YAML file and parameters during the initialization, the latter will overwrite the configuration item provided by the YAML function.
You can use `show_current_configs` function to show the current configuration of the network:

In [13]:
network = AifNet("./pre_trained/single_parameter/32/diffusion/network_configs.yaml")
#network = AifNet("./pre_trained/single_parameter/32/diffusion/network_configs.yaml",condition_layers=[-3])
network.show_current_configs()

attention_layers: [2, 3]
condition_layers: [-2]
depth_each_layer: 2
dim_basic: 16
dim_condition: 3
dim_encoded_time: 8
dim_in: 6
dim_multipliers: [1, 2, 4, 4]
dim_out: 3
heads_attention: 4
linear_attention: False
skip_connection_scale: 0.707
use_input_condition: True
condition_dim: 0


We provide several trainer classes for training procedures that can be easily invoked. You can also use `show_config_options` function to see all the possible configurations of the trainer:

In [14]:
diffusion_trainer=DiffusionTrainer()
diffusion_trainer.show_config_options()

Mandatory Configuration:
    name (str): Name of the training.
    save_path (str): Path to save the training results.
    batch_size_train (int): Batch size for training.
    epochs (int): Number of epochs for training.
    lr (float): Initial learning rate.

Optional Configuration:
    device (str, default value: cpu): Device for training.
    random_seed (int): Random seed for training. Default is the same as batch_size_train.
    batch_size_validation (int): Batch size for validation. Default is the same as batch_size_train.
    shuffle_train (bool, default value: True): Whether to shuffle the training dataset.
    shuffle_validation (bool): Whether to shuffle the validation dataset. Default is the same as shuffle_train.
    num_workers_train (int, default value: 0): Number of workers for training.
    num_workers_validation (int): Number of workers for validation. Default is the same as num_workers_train.
    validation_epoch_frequency (int, default value: 1): Frequency of validat

You can use `train_from_scratch` function to train a network from scratch or `train_from_checkpoint` function to train from a checkpoint. Similarly, you can also give a YAML file or directly set the configurations in the function.

Our trainer provides some useful features like checkpoint saving, loss recorder with TensorBoard, and configuration saving. You can also utilize the `TrainedProject` class we provided to manage the trained project. Details can be found in airfoil_diffusion/trainer_base.py and airfoil_diffusion/trainer.py. Explore by yourself and enjoy!

Now, let's train our networks!

* Train the diffusion model:

In [ ]:
diffusion_trainer.train_from_scratch(name="diffusion",
                                     network=network,
                                     train_dataset=train_dataset,
                                     path_config_file="./pre_trained/train_configs.yaml",
                                     save_path="./training/single_parameter/32/",)

* Train heteroscedastic model:

In [ ]:
heteroscedastic_trainer=HeteroscedasticTrainer()
network = AifNet("./pre_trained/single_parameter/32/heteroscedastic/network_configs.yaml")
heteroscedastic_trainer.train_from_scratch(name="heteroscedastic",
                                            network=network,
                                            train_dataset=train_dataset,
                                            path_config_file="./pre_trained/train_configs.yaml",
                                            save_path="./training/single_parameter/32/",)

* Train BNNs with different KL scaling:

Note that you need to install baysian_torch (https://github.com/IntelLabs/bayesian-torch) first. We used version 0.3.0 for examples here and cases in the manuscript.

In [ ]:
from bayesian_torch.models.dnn_to_bnn import dnn_to_bnn
bnn_trainer=BNNTrainer()
const_bnn_prior_parameters = read_configs("./pre_trained/single_parameter/32/BNN/bnn_configs.yaml")

for KL_scaling in [0.0001,0.001,0.01]:
    network = AifNet("./pre_trained/single_parameter/32/BNN/network_configs.yaml")
    dnn_to_bnn(network, const_bnn_prior_parameters)
    bnn_trainer.train_from_scratch(name="BNN_{}".format(KL_scaling),
                                    network=network,
                                    train_dataset=train_dataset,
                                    path_config_file="./pre_trained/train_configs.yaml",
                                    save_path="./training/single_parameter/32/",
                                    KL_scaling=KL_scaling)